## Barbell chart - end of life care by service line

In [1]:
import pandas as pd
import numpy as np
import altair as alt
pd.options.mode.chained_assignment = None

### Chose a focus hospital and get a benchmark group for comparison

In [2]:
focusHospital = 60307  #deid hosptial number
focusName = 'General Hospital'

In [3]:
#see post "Hospital benchmarking" for details
import getBenchmarkGroup 
compareGroupList = getBenchmarkGroup.getBenchmarkGroup(hospitalId=focusHospital, groupSize=50)
print(compareGroupList)

[90082, 30649, 50436, 30595, 30681, 40565, 30527, 30699, 50025, 20394, 80040, 80150, 80010, 20153, 20003, 80138, 20303, 70575, 50062, 50363, 10071, 90406, 60141, 20085, 40334, 60397, 40582, 20069, 40491, 80124, 40495, 20190, 20177, 70655, 40579, 90298, 90164, 40072, 70437, 90110, 90525, 60210, 40445, 20329, 10130, 60044, 30420, 80200, 50126]


In [4]:
#load prepared data
eolDf = pd.read_excel('..\\reference\\eolData.xlsx')

focusDf = eolDf.loc[eolDf['HOSP_NIS']==focusHospital]
focusDf['Level'] = 'Focus'
focusDf['DNRorPC'] = focusDf['DNR'] + focusDf['PC']
#get top 10 service lines of interest
slList = focusDf.sort_values('DNRorPC',ascending=False)['SL'].tolist()[:10]
focusDf = focusDf.loc[focusDf['SL'].isin(slList)]

compareDf = eolDf.loc[eolDf['HOSP_NIS'].isin(compareGroupList)]
compareDf = compareDf.groupby('SL').sum().reset_index()
compareDf = compareDf.loc[compareDf['SL'].isin(slList)]
compareDf['Level'] = 'Compare'

In [5]:
eol = focusDf.append(compareDf)
eol[['Level','SL','Cases','DNR','PC']].head()

,Level,SL,Cases,DNR,PC
65971,Focus,CARDIOLOGY,824,39,5
65973,Focus,ENDOCRINOLOGY,195,14,3
65974,Focus,GASTROENTEROLOGY,500,22,5
65975,Focus,GENERAL MEDICINE,301,35,4
65976,Focus,GENERAL SURGERY,292,8,1


### Prepare data for Altair chart

In [6]:
for col in ['DNR','PC']:
    eol[f'{col} %'] = (eol[f'{col}']/eol['Cases']) * 100

eol['SL'] = pd.Categorical(eol['SL'], slList) #order top 10 service lines

eol = eol.sort_values(['SL','Level'])
eol = pd.melt(eol, id_vars=['Level', 'SL'], value_vars=[c for c in eol.columns if '%' in c])

eolChart=pd.DataFrame()
for sl in eol['SL'].unique():
    for cat in eol['variable'].unique():
        temp = eol.loc[(eol['SL']==sl) & (eol['variable']==cat)]
        temp['% Focus'] = temp.loc[temp['Level']=='Focus']['value'].values[0]
        temp['% Compare'] = temp.loc[temp['Level']=='Compare']['value'].values[0]
        temp.loc[temp['% Focus'] - temp['% Compare'] > 0 , 'Color compare'] = \
                                                    f'{focusName} above benchmark'
        temp.loc[temp['% Focus'] - temp['% Compare'] < 0, 'Color compare'] = \
                                                    f'{focusName} below benchmark'
        temp['Color compare'] =  temp['Color compare'].fillna('Benchmark')
        temp.loc[temp['Level']=='Compare', 'Color compare'] = 'Benchmark'
        eolChart = eolChart.append(temp)
eolChart.head()

,Level,SL,variable,value,% Focus,% Compare,Color compare
0,Compare,PULMONARY,DNR %,6.411180,11.478599,6.411180,Benchmark
1,Focus,PULMONARY,DNR %,11.478599,11.478599,6.411180,General Hospital above benchmark
20,Compare,PULMONARY,PC %,2.161072,0.972763,2.161072,Benchmark
21,Focus,PULMONARY,PC %,0.972763,0.972763,2.161072,General Hospital below benchmark
2,Compare,ONCOLOGY/HEMATOLOGY,DNR %,5.918004,18.012422,5.918004,Benchmark


### Use layering and faceting for final barbell-style plot

In [7]:
eolChart['SL'] = eolChart['SL'].str.title()
slCapitalList = [sl.title() for sl in slList]
eolChart['variable'] = eolChart['variable'].replace({'DNR %':'DNR','PC %': 'Palliative care'})

bells = alt.Chart(eolChart).mark_circle(size=250,opacity=1,stroke='#696969',strokeWidth=.5).encode(
        x=alt.X('value:Q',scale=alt.Scale(zero=False),axis=alt.Axis(grid=True,labelAngle=0,
                    title='% Cases',domainWidth=1,tickMinStep=1)),
        y=alt.Y('SL:N',sort=slCapitalList,title=None,axis=alt.Axis(domainWidth=0)), 
        color=alt.Color('Color compare',
                        scale=alt.Scale(domain=['Benchmark', 
                                     f'{focusName} above benchmark', f'{focusName} below benchmark'], 
                        range=['#696969','#33a02c','#e31a1c'])))

bar = alt.Chart(eolChart).mark_rule(color='#c0c0c0',size=3).encode(alt.X('% Focus:Q'),
                        alt.X2('% Compare:Q'), y=alt.Y('SL:N',sort=slCapitalList)) 

alt.layer(bar+bells).properties(width=310,height=420
                    ).facet(alt.Column('variable:N',title=None,sort=['DNR','Palliative care'])  
                    ).resolve_scale(x='independent'
                    ).properties(title=f'{focusName} - end of life care by service line'
                    ).configure_header(labelFontStyle='bold',labelColor='#6e6e6e',labelFont='Arial',
                                       labelFontSize=16,labelPadding=12
                    ).configure_view(strokeWidth=0,fill='#f2f2f2'
                    ).configure_axis(labelColor='#6e6e6e',labelFont='Arial',labelFontSize=14,
                                     titleFont='Arial',titleFontSize=12,titleColor='#6e6e6e',
                                     labelLimit=1000,tickSize=0
                    ).configure_axisY(labelPadding=160,labelAlign='left'
                    ).configure_legend(labelColor='#6e6e6e',labelFont='Arial',labelFontSize=14,
                                     title=None,orient='none',labelLimit=250,direction='horizontal',
                                     legendX=35,legendY=470
                    ).configure_title(fontSize=20,color='#6e6e6e',font='Arial',anchor='start',
                                      offset=25)

alt.FacetChart(...)